In [1]:
import pandas as pd
import chess
import chess.pgn
import chess.engine
import asyncio

In [2]:
eng = chess.engine.SimpleEngine.popen_uci("../../stockfish/stockfish")

In [3]:
p = open("../data/RuyLopezMarshall.pgn")

In [4]:
games_id = []
games = []

while True:
    offset = p.tell()
    headers = chess.pgn.read_headers(p)
    if headers is None:
        break
    games_id.append(offset)

for offset in games_id:
    p.seek(offset)
    games.append(chess.pgn.read_game(p))

In [5]:
dados = []
count = -1

for game in games:
    count += 1
    print('{0}\t\t'.format(count), end='\r')
    row = {}
    head = game.headers

    if head['Site'].endswith(' INT') or head['Site'] == '?':
        continue

    row['id'] = games_id[count]
    row['Event'] = head['Event']
    row['Site'] = head['Site']
    row['Date'] = head['Date'].split('.')[0]
    row['White'] = head['White']
    row['Black'] = head['Black']
    row['Result'] = head['Result']
    row['BlackElo'] = head['BlackElo']
    row['ECO'] = 'C89'
    row['WhiteElo'] = head['WhiteElo']
    row['White Short Castle'] = 0
    row['White Long Castle'] = 0
    row['Black Short Castle'] = 0
    row['Black Long Castle'] = 0
    row['Opposite side castling'] = 0
    row['Evals'] = []

    # quem ganhou
    if head['Result'] == '0-1':
        row['WhiteWon'] = 0
        row['BlackWon'] = 1
    elif head['Result'] == '1-0':
        row['WhiteWon'] = 1
        row['BlackWon'] = 0
    else:
        row['WhiteWon'] = 0
        row['BlackWon'] = 0

    # qntd de moves
    final_position = game[0].end().board()
    row['Moves'] = final_position.fullmove_number

    # qntd de peças restantes
    final_fen = final_position.board_fen()
    white_count = 0
    black_count = 0
    white_pieces = ['P', 'N', 'B', 'R', 'Q']
    black_pieces = ['p', 'n', 'b', 'r', 'q']
    white_has_queen = 0
    black_has_queen = 0

    for char in range(0, len(final_fen)):
        if final_fen[char] in white_pieces:
            white_count += 1
        elif final_fen[char] in black_pieces:
            black_count += 1

        if final_fen[char] == 'Q':
            white_has_queen = 1
        elif final_fen[char] == 'q':
            black_has_queen = 1

    row['WhiteFinalPiecesCount'] = white_count
    row['BlackFinalPiecesCount'] = black_count
    row['FinalPiecesCount'] = white_count + black_count

    if white_has_queen == 0 and black_has_queen == 0:
        row['QueenTrade'] = 1
    else:
        row['QueenTrade'] = 0

    board = game[0].board()
    info = eng.analyse(board, chess.engine.Limit(time=0.1))
    row['Evals'].append(info['score'].white().score(mate_score=100000))

    for move in game[0].mainline_moves():        
        board.push(move)
        info = eng.analyse(board, chess.engine.Limit(time=0.1))
        row['Evals'].append(info['score'].white().score(mate_score=100000))

        m = move.uci()

        flag_wking_move = 0
        flag_bking_move = 0

        if m[:2] == 'e1' and flag_wking_move == 0:
            flag_wking_move = 1

            if m == 'e1g1':
                row['White Short Castle'] = 1
            elif m == 'e1c1':
                row['White Long Castle'] = 1

        elif m[:2] == 'e8' and flag_bking_move == 0:
            flag_bking_move = 1

            if m == 'e8g8':
                row['Black Short Castle'] = 1
            elif m == 'e8c8':
                row['Black Long Castle'] = 1

        if (row['White Short Castle'] == 1 and row['Black Long Castle'] == 1) or \
                (row['Black Short Castle'] == 1 and row['White Long Castle'] == 1):
            row['Opposite side castling'] = 1
    
    dados.append(row)

eng.quit()
df = pd.DataFrame(dados)


In [6]:
# corrigir places/Sites
def string_analyzer(row):
    s = row['Site']
    splitted_s = s.split(' ')
    first_part = splitted_s[:-1]
    last_part = splitted_s[-1]
    city = ''

    if last_part.isupper() and len(splitted_s) > 1:
        for w in first_part:
            city += w + ' '
    else:
        city = s

    row['Site'] = city

    return row


df = df.apply(lambda row: string_analyzer(row=row), axis=1)


In [7]:
df.to_csv('../data/df_jogos.csv', index=False)
df

,id,Event,Site,Date,White,Black,Result,BlackElo,ECO,WhiteElo,...,Black Long Castle,Opposite side castling,Evals,WhiteWon,BlackWon,Moves,WhiteFinalPiecesCount,BlackFinalPiecesCount,FinalPiecesCount,QueenTrade
0,0,New York,New York,1918,"Morrison, John Stuart","Marshall, Frank James",0-1,,C89,,...,0,0,"[38, 54, 25, 45, 37, 61, 50, 51, 35, 53, 43, 6...",0,1,85,1,3,4,0
1,1168,New York,New York,1918,"Capablanca, Jose Raul","Marshall, Frank James",1-0,,C89,,...,0,0,"[35, 43, 27, 39, 34, 33, 50, 50, 54, 51, 43, 5...",1,0,36,7,5,12,0
2,1772,Brno,Brno,1921,"Sery, S.","Vecsey, Zoltan",1-0,,C89,,...,0,0,"[30, 54, 55, 57, 37, 50, 56, 62, 56, 72, 66, 6...",1,0,17,12,10,22,0
3,2129,NLD-ch,Nijmegen,1921,"Te Kolste, Jan Willem","Loman, Rudolf",0-1,,C89,,...,0,0,"[37, 48, 46, 46, 55, 56, 49, 53, 46, 69, 67, 5...",0,1,19,11,10,21,0
4,2522,USA sim,USA,1926,"Lasker, Emanuel","Bigelow, Horace Ransom",0-1,,C89,,...,0,0,"[26, 50, 52, 50, 50, 55, 46, 46, 47, 57, 53, 5...",0,1,20,11,11,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,2113381,23rd European Teams,Terme Catez,2021,"Agopov,M","Ragger,M",1/2-1/2,2668,C89,2430,...,0,0,"[26, 28, 27, 38, 46, 50, 28, 51, 46, 67, 59, 6...",0,0,55,2,1,3,1
2980,2117758,8th Sunway Sitges 2021,Sitges,2021,"Kurmann,O","Van Dael,Siem",1-0,2300,C89,2466,...,0,0,"[25, 50, 41, 39, 35, 63, 54, 60, 46, 64, 53, 5...",1,0,51,8,5,13,0
2981,2118961,7th Gashimov Mem Blitz,Baku,2021,"Karjakin,Sergey","Caruana,F",1/2-1/2,2792,C89,2743,...,0,0,"[24, 37, 35, 52, 46, 51, 29, 69, 45, 74, 54, 6...",0,0,37,4,4,8,1
2982,2119588,World Rapid Women 2021,Warsaw,2021,"Loskutova,Viktoriya","Abdumalik,Z",1-0,2491,C89,2100,...,0,0,"[34, 54, 45, 40, 49, 43, 51, 41, 58, 59, 42, 4...",1,0,42,6,5,11,1
